Run this cell to initialize population synthesis

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from synthesize import *
from preprocess import *


config_folder = "../configs/Chicago"
n = 1000

person = process_pums_data(config_folder)
population_sample = synthesize_population(
    config_folder=config_folder, n_sample=n, min_age=18, max_age=65, random_state=2
)
ploc = puma_locations(config_folder)

Run this cell to initalize template generation

In [16]:
smg = SystemMessageGenerator(config_folder, "SystemMessage.j2", verbose_debug=False, shuffle=False, wrap=80)
year = 2019

def wrap_text(text, n):
    """
    Utility to break text on char width
    """
    words = text.split()
    lines = []
    current_line = []

    for word in words:
        if sum(len(w) for w in current_line) + len(current_line) + len(word) <= n:
            current_line.append(word)
        else:
            # Join current line and start a new one
            lines.append(' '.join(current_line))
            current_line = [word]

    # Add last line if not empty
    if current_line:
        lines.append(' '.join(current_line))

    return '\n'.join(lines)

output = []
attribute_descriptions = get_attribute_descriptions(person)
for i, individual in population_sample.reset_index().iterrows():
    individual_attributes = attribute_decoder_dict(individual.to_dict(), person)
    system_message = smg.write_system_message(**individual_attributes, **attribute_descriptions, ploc=ploc, YEAR=year)
    print(system_message)
    print('\n')

You are a 51 year old woman of Polish ancestry that lives in Merrionette Park,
Illinois. You were born in 1968 in Canada and became a U.S. citizen by
naturalization in 2006. You live alone. Your highest educational attainment is a
Master's degree in English Language And Literature. You are a Local government
employee in elementary and secondary schools. In the last year you earned
$78,000. You typically work 40 hours per week and have worked for 52 weeks in
the past year.


You are a 50 year old man of Mexican ancestry that lives in Evergreen Park,
Illinois. You were born in the U.S. in 1969 in Illinois. You have been married
for 11 years. Your highest educational attainment is an Associate's degree. You
are an employee of a for-profit business in sporting and athletic goods, and
doll, toy and game. In the last year you earned $69,000. You typically work 40
hours per week and have worked for 52 weeks in the past year.


You are a 42 year old woman of Croatian ancestry that lives in Als

In [6]:
system_message

"\nYou\nare\na\n45\nyear\nold\nman\nof\nItalian\nancestry\nthat\nlives\nin\nthe\nChicago\nmetropolitan\narea\nof\nIllinois.\nYou\nwere\nborn\nin\nthe\nU.S.\nin\n1974\nin\nIllinois.\nYou\nhave\nbeen\nmarried\nfor\n1\nyears\nand\nlive\nwith\na\nbiological\nson\nor\ndaughter.\nYour\nhighest\neducational\nattainment\nis\nan\nAssociate's\ndegree.\nYou\nare\na\nLocal\ngovernment\nemployee\nin\nelementary\nand\nsecondary\nschools.\nIn\nthe\nlast\nyear\nyou\nearned\n$48,000.\nYou\ntypically\nwork\n40\nhours\nper\nweek\nand\nhave\nworked\nfor\n52\nweeks\nin\nthe\npast\nyear."